In [ ]:
# coding: utf-8

from SimilarityClustering import SimilarityClustering
import articles_data

from gensim.models import Word2Vec
import numpy as np
import pandas as pd
import cnouns as cn
from pymongo import MongoClient
import datetime

client = MongoClient('mongodb://localhost:27017/somanews')
client.somanews.authenticate('ssomanews', 'ssomanews1029')
db = client.get_database('somanews')

crawled_collection = db.get_collection('crawledArticles')
clusters_collection = db.get_collection('bclusters')
articles_collection = db.get_collection('barticles')

datastore_dir = "../datastore/"
catelist_path = datastore_dir + "category2.p"
w2v_src_dir = datastore_dir + "w2v_src3"
w2v_path = datastore_dir + "sejongcorpus_w2v3_2.p"

corpus_path = datastore_dir + "corpus2.p"
target_time = datetime.datetime.now()
prefix = int("%.2d%.2d"%(target_time.month, target_time.day))
prefix_str = "%d_00" % prefix

dicts = [u'새누리', u'새누리당', u'더민주', u'더민주당', u'최순실', u'박대통령', u'국회의장', u'야권의요구', u'정기국회', u'참여정부']
def tokenizer(inp_str):
    return cn.custom_pos_tags(inp_str, dicts)

# # Model
train_df = articles_data.find_recent_articles(crawled_collection, catelist_path, target_time)
sc = SimilarityClustering()
sc.train("cate", w2v_path, train_df, path=datastore_dir, prefix=prefix_str, tokenizer=tokenizer,
            threshold=0.7,
            cnt_threshold=2,
            repeat=1,
            model_name='dbow+dmm')

# # Save
sc.save(path=datastore_dir, prefix=prefix_str)
calced_clusters = sc.save_to_db(prefix, clusters_collection, articles_collection, target_time, test=True)

Tokenizing: 2457it [00:40, 61.18it/s]


Size of documents: 2457
Loading word2vec model.....
Complete to load.


In [2]:
sc = SimilarityClustering.load(path=datastore_dir, prefix=prefix_str, model_name='dbow+dmm', only_d2v=False)

total: 2317 , size_1: 2199 , countby: 0
distance: 0.0
variance: 0.0
similarity: nan
cohesion: 0.0
in_threshold: 0.0
time_mean: 0.0
time_v: 0.0
portion: 0.0


In [3]:
sc.iner_score(threshold=0.7, cnt_threshold=10)

total: 2317 , size_1: 2199 , countby: 0
distance: 0.0
variance: 0.0
similarity: nan
cohesion: 0.0
in_threshold: 0.0
time_mean: 0.0
time_v: 0.0
portion: 0.0


In [4]:
sc.print_clusters(top=15, sortby='similarity')

In [5]:
cdf = pd.DataFrame(calced_clusters, columns=['cluster', 'portion', 'deltaTime', 'cohesion', 'ntc'])
sort_cdf = cdf.sort_values('ntc', ascending=False)
sort_cdf

,cluster,portion,deltaTime,cohesion,ntc
15,1114644,0.109091,0.489065,0.878748,29.343758
10,1114061,0.054545,0.570149,0.929216,26.885984
6,1115635,0.054545,0.666372,0.937820,24.439374
11,1115786,0.054545,0.835447,0.926113,21.285313
2,1114716,0.054545,0.851397,0.945806,21.258016
8,1114635,0.054545,0.870340,0.931753,20.861839
1,1114606,0.054545,0.932257,0.953328,20.314481
3,1115565,0.054545,0.942967,0.944685,20.106225
0,1114187,0.054545,0.960603,0.954372,20.008391
7,1114659,0.054545,0.951397,0.933531,19.900720


In [6]:
for idx, row in sort_cdf.iterrows():
    for c in calced_clusters:
        if(c['cluster'] == row.cluster):
            print c['ntc'], c['leading']['title']

29.3437579345 민주노총, 경찰에 11월 12일 청와대 앞까지 행진 신고
26.885984293 [경제계 인사]대신증권 外
24.4393738689 ‘수백억 횡령’ 부산 엘시티 이영복 회장 검거
21.2853134675 청와대 "박 대통령 '세월호 7시간 성형시술' 의혹, 근거없는 유언비어"
21.2580162163 광주~원주 제2영동고속도로 11일 개통
20.861838577 ‘국정농단 비판’ 학생들 대자보 옆에 답장 쓴 선생님
20.3144814923 ‘콜센터 상담원’ 등 감정노동자 전문 지원기관 생긴다
20.1062245098 대우조선 ‘밑 빠진 독에 물 붓기’…산은·금융위 감자로 수천억 손실
20.008391064 두산밥캣 공모가 3만원 확정···8일부터 일반 청약
19.9007196187 ‘전두환 아들에 20억 갈취’ 범서방파 무더기 검거
19.8880125993 “박근혜 퇴진” 벌써부터 시민 물결 가득…경찰 추산 11만명 돌파
19.6703308744 [11·12 민중총궐기]"이게 나라냐", '2016 민중총궐기'는 오체투지로 시작
19.5688467845 제주에서도 1천여 학생·시민들 ‘박근혜 하야’ 외쳐
19.5451367335 검찰, 삼성전자 압수수색…최순실·정유라 특혜지원 의혹
19.3631432829 [11·12 민중총궐기]총궐기 본집회 시작, 주최측 “25만명” 운집
19.2248137244 [긴급]김무성, 긴급회견 열어 “朴 대통령에 탈당, 총리 지명 철회 공식 요구”
